# media: Iceberg and Liquid Clustering Demo


## Overview


This notebook demonstrates the power of **Iceberg and Liquid Clustering** in Oracle AI Data Platform (AIDP) Workbench using a media and entertainment analytics use case. Liquid clustering automatically optimizes data layout for query performance without requiring manual partitioning or Z-Ordering.

### What is Iceberg?

Apache Iceberg is an open table format for huge analytic datasets that provides:

- **Schema evolution**: Add, drop, rename, update columns without rewriting data
- **Partition evolution**: Change partitioning without disrupting queries
- **Time travel**: Query historical data snapshots for auditing and rollback
- **ACID transactions**: Reliable concurrent read/write operations
- **Cross-engine compatibility**: Works with Spark, Flink, Presto, Hive, and more
- **Open ecosystem**: Apache 2.0 licensed, community-driven development

### Delta Universal Format with Iceberg

Delta Universal Format enables Iceberg compatibility while maintaining Delta's advanced features like liquid clustering. This combination provides:

- **Best of both worlds**: Delta's performance optimizations with Iceberg's openness
- **Multi-engine access**: Query the same data from different analytics engines
- **Future-proof architecture**: Standards-based approach for long-term data investments
- **Enhanced governance**: Rich metadata and catalog integration

### What is Liquid Clustering?

Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:

- **Automatic optimization**: No manual tuning required
- **Improved query performance**: Faster queries on clustered columns
- **Reduced maintenance**: No need for manual repartitioning
- **Adaptive clustering**: Adjusts as data patterns change

### Use Case: Content Performance and User Engagement Analytics

We'll analyze media content consumption and user engagement data. Our clustering strategy will optimize for:

- **User-specific queries**: Fast lookups by user ID
- **Time-based analysis**: Efficient filtering by viewing and engagement dates
- **Content performance patterns**: Quick aggregation by content type and engagement metrics

### AIDP Environment Setup

This notebook leverages the existing Spark session in your AIDP environment.

In [1]:
# Create media catalog and analytics schema

# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS media")

spark.sql("CREATE SCHEMA IF NOT EXISTS media.analytics")

print("Media catalog and analytics schema created successfully!")

Media catalog and analytics schema created successfully!


## Step 2: Create Delta Table with Liquid Clustering

### Table Design

Our `content_engagement_uf` table will store:

- **user_id**: Unique user identifier
- **engagement_date**: Date and time of engagement
- **content_type**: Type (Video, Article, Podcast, Live Stream)
- **watch_time**: Time spent consuming content (minutes)
- **content_id**: Specific content identifier
- **engagement_score**: User engagement metric (0-100)
- **device_type**: Device used (Mobile, Desktop, TV, etc.)

### Clustering Strategy

We'll cluster by `user_id` and `engagement_date` because:

- **user_id**: Users consume multiple pieces of content, grouping their viewing history together
- **engagement_date**: Time-based queries are critical for content performance analysis, recommendation systems, and user behavior trends
- This combination optimizes for both personalized content recommendations and temporal engagement analysis

In [1]:
# Create Delta table with liquid clustering

# CLUSTER BY defines the columns for automatic optimization
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
data_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("engagement_date", TimestampType(), True),
    StructField("content_type", StringType(), True),
    StructField("watch_time", DoubleType(), True),
    StructField("content_id", StringType(), True),
    StructField("engagement_score", IntegerType(), True),
    StructField("device_type", StringType(), True)
])
spark.sql("""

CREATE TABLE IF NOT EXISTS media.analytics.content_engagement_uf (
    user_id STRING,
    engagement_date TIMESTAMP,
    content_type STRING,
    watch_time DECIMAL(8,2),
    content_id STRING,
    engagement_score INT,
    device_type STRING
)

USING DELTA

TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (user_id, engagement_date)

""")

print("Delta table with Iceberg compatibility and liquid clustering created successfully!")

print("Universal format enables Iceberg features while CLUSTER BY (columns) optimizes data layout.")

Delta table with Iceberg compatibility and liquid clustering created successfully!
Universal format enables Iceberg features while CLUSTER BY (columns) optimizes data layout.


## Step 3: Generate Media Sample Data

### Data Generation Strategy

We'll create realistic media engagement data including:

- **12,000 users** with multiple content interactions over time
- **Content types**: Video, Article, Podcast, Live Stream
- **Realistic engagement patterns**: Peak viewing times, content preferences, device usage
- **Engagement metrics**: Watch time, completion rates, interaction scores

### Why This Data Pattern?

This data simulates real media scenarios where:

- User preferences drive content recommendations
- Engagement metrics determine content success
- Device usage affects viewing experience
- Time-based patterns influence programming decisions
- Personalization requires historical user behavior

In [1]:
# Generate sample media engagement data

# Using fully qualified imports to avoid conflicts

import random

from datetime import datetime, timedelta


# Define media data constants

CONTENT_TYPES = ['Video', 'Article', 'Podcast', 'Live Stream']

DEVICE_TYPES = ['Mobile', 'Desktop', 'Tablet', 'Smart TV', 'Gaming Console']

# Base engagement parameters by content type

ENGAGEMENT_PARAMS = {

    'Video': {'avg_watch_time': 15, 'engagement_base': 75, 'frequency': 12},

    'Article': {'avg_watch_time': 8, 'engagement_base': 65, 'frequency': 8},

    'Podcast': {'avg_watch_time': 25, 'engagement_base': 70, 'frequency': 6},

    'Live Stream': {'avg_watch_time': 45, 'engagement_base': 80, 'frequency': 4}

}

# Device engagement multipliers

DEVICE_MULTIPLIERS = {

    'Mobile': 0.9, 'Desktop': 1.0, 'Tablet': 0.95, 'Smart TV': 1.1, 'Gaming Console': 1.05

}


# Generate content engagement records

engagement_data = []

base_date = datetime(2024, 1, 1)


# Create 12,000 users with 10-40 engagement events each

for user_num in range(1, 12001):

    user_id = f"USER{user_num:06d}"
    
    # Each user gets 10-40 engagement events over 12 months

    num_engagements = random.randint(10, 40)
    
    for i in range(num_engagements):

        # Spread engagements over 12 months

        days_offset = random.randint(0, 365)

        engagement_date = base_date + timedelta(days=days_offset)
        
        # Add realistic timing (more engagement during certain hours)

        hour_weights = [2, 1, 1, 1, 1, 1, 3, 6, 8, 7, 6, 7, 8, 9, 10, 9, 8, 10, 12, 9, 7, 5, 4, 3]

        hours_offset = random.choices(range(24), weights=hour_weights)[0]

        engagement_date = engagement_date.replace(hour=hours_offset, minute=random.randint(0, 59), second=0, microsecond=0)
        
        # Select content type

        content_type = random.choice(CONTENT_TYPES)

        params = ENGAGEMENT_PARAMS[content_type]
        
        # Select device type

        device_type = random.choice(DEVICE_TYPES)

        device_multiplier = DEVICE_MULTIPLIERS[device_type]
        
        # Calculate watch time with variations

        time_variation = random.uniform(0.3, 2.5)

        watch_time = round(params['avg_watch_time'] * time_variation * device_multiplier, 2)
        
        # Content ID

        content_id = f"{content_type[:3].upper()}{random.randint(10000, 99999)}"
        
        # Engagement score (based on content type, device, and some randomness)

        engagement_variation = random.randint(-15, 15)

        engagement_score = max(0, min(100, int(params['engagement_base'] * device_multiplier) + engagement_variation))
        
        engagement_data.append({

            "user_id": user_id,

            "engagement_date": engagement_date,

            "content_type": content_type,

            "watch_time": watch_time,

            "content_id": content_id,

            "engagement_score": engagement_score,

            "device_type": device_type

        })



print(f"Generated {len(engagement_data)} content engagement records")

print("Sample record:", engagement_data[0])

Generated 300771 content engagement records
Sample record: {'user_id': 'USER000001', 'engagement_date': datetime.datetime(2024, 11, 25, 9, 29), 'content_type': 'Video', 'watch_time': 25.22, 'content_id': 'VID16142', 'engagement_score': 77, 'device_type': 'Smart TV'}


## Step 4: Insert Data Using PySpark

### Data Insertion Strategy

We'll use PySpark to:

1. **Create DataFrame** from our generated data
2. **Insert into Delta table** with liquid clustering
3. **Verify the insertion** with a sample query

### Why PySpark for Insertion?

- **Distributed processing**: Handles large datasets efficiently
- **Type safety**: Ensures data integrity
- **Optimization**: Leverages Spark's query optimization
- **Liquid clustering**: Automatically applies clustering during insertion

In [1]:
# Insert data using PySpark DataFrame operations

# Using fully qualified function references to avoid conflicts


# Create DataFrame from generated data

df_engagement = spark.createDataFrame(engagement_data, schema=data_schema)


# Display schema and sample data

print("DataFrame Schema:")

df_engagement.printSchema()



print("\nSample Data:")

df_engagement.show(5)


# Insert data into Delta table with liquid clustering

# The TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (user_id, engagement_date) will automatically optimize the data layout

df_engagement.write.mode("overwrite").insertInto("media.analytics.content_engagement_uf")


print(f"\nSuccessfully inserted {df_engagement.count()} records into media.analytics.content_engagement_uf")

print("Liquid clustering automatically optimized the data layout during insertion!")

DataFrame Schema:
root
 |-- user_id: string (nullable = true)
 |-- engagement_date: timestamp (nullable = true)
 |-- content_type: string (nullable = true)
 |-- watch_time: double (nullable = true)
 |-- content_id: string (nullable = true)
 |-- engagement_score: integer (nullable = true)
 |-- device_type: string (nullable = true)


Sample Data:
+----------+-------------------+------------+----------+----------+----------------+--------------+
|   user_id|    engagement_date|content_type|watch_time|content_id|engagement_score|   device_type|
+----------+-------------------+------------+----------+----------+----------------+--------------+
|USER000001|2024-11-25 09:29:00|       Video|     25.22|  VID16142|              77|      Smart TV|
|USER000001|2024-06-25 22:51:00|     Podcast|     19.84|  POD74343|              80|Gaming Console|
|USER000001|2024-05-19 17:06:00| Live Stream|     89.71|  LIV77628|              79|Gaming Console|
|USER000001|2024-07-28 21:15:00|     Podcast|     23.


Successfully inserted 300771 records into media.analytics.content_engagement_uf
Liquid clustering automatically optimized the data layout during insertion!


## Step 5: Demonstrate Liquid Clustering Benefits

### Query Performance Analysis

Now let's see how liquid clustering improves query performance. We'll run queries that benefit from our clustering strategy:

1. **User engagement history** (clustered by user_id)
2. **Time-based content analysis** (clustered by engagement_date)
3. **Combined user + time queries** (optimal for our clustering)

### Expected Performance Benefits

With liquid clustering, these queries should be significantly faster because:

- **Data locality**: Related records are physically grouped together
- **Reduced I/O**: Less data needs to be read from disk
- **Automatic optimization**: No manual tuning required

In [1]:
# Demonstrate liquid clustering benefits with optimized queries


# Query 1: User engagement history - benefits from user_id clustering

print("=== Query 1: User Engagement History ===")

user_history = spark.sql("""

SELECT user_id, engagement_date, content_type, watch_time, engagement_score

FROM media.analytics.content_engagement_uf

WHERE user_id = 'USER000001'

ORDER BY engagement_date DESC

LIMIT 10

""")



user_history.show()

print(f"Records found: {user_history.count()}")



# Query 2: Time-based high-engagement content analysis - benefits from engagement_date clustering

print("\n=== Query 2: Recent High-Engagement Content ===")

high_engagement = spark.sql("""

SELECT engagement_date, user_id, content_id, content_type, engagement_score, watch_time

FROM media.analytics.content_engagement_uf

WHERE DATE(engagement_date) = '2024-02-15' AND engagement_score > 85

ORDER BY engagement_score DESC, watch_time DESC

""")



high_engagement.show()

print(f"High-engagement records found: {high_engagement.count()}")



# Query 3: Combined user + time query - optimal for our clustering strategy

print("\n=== Query 3: User Content Preferences ===")

user_preferences = spark.sql("""

SELECT user_id, engagement_date, content_type, watch_time, device_type

FROM media.analytics.content_engagement_uf

WHERE user_id LIKE 'USER000%' AND engagement_date >= '2024-02-01'

ORDER BY user_id, engagement_date

LIMIT 25

""")



user_preferences.show()

print(f"User preference records found: {user_preferences.count()}")

=== Query 1: User Engagement History ===


+----------+-------------------+------------+----------+----------------+
|   user_id|    engagement_date|content_type|watch_time|engagement_score|
+----------+-------------------+------------+----------+----------------+
|USER000001|2024-12-11 13:14:00| Live Stream|     37.69|              84|
|USER000001|2024-12-01 11:34:00|     Podcast|     23.76|              71|
|USER000001|2024-11-25 09:29:00|       Video|     25.22|              77|
|USER000001|2024-10-31 11:12:00|     Podcast|     44.24|              78|
|USER000001|2024-09-30 20:12:00|     Article|      9.16|              60|
|USER000001|2024-07-28 21:15:00|     Podcast|     23.12|              58|
|USER000001|2024-07-04 08:19:00|     Article|     17.52|              57|
|USER000001|2024-06-25 22:51:00|     Podcast|     19.84|              80|
|USER000001|2024-06-24 00:50:00| Live Stream|     39.48|              88|
|USER000001|2024-05-19 17:06:00| Live Stream|     89.71|              79|
+----------+-------------------+------

Records found: 10

=== Query 2: Recent High-Engagement Content ===


+-------------------+----------+----------+------------+----------------+----------+
|    engagement_date|   user_id|content_id|content_type|engagement_score|watch_time|
+-------------------+----------+----------+------------+----------------+----------+
|2024-02-15 13:16:00|USER001757|  LIV68728| Live Stream|             100|    116.08|
|2024-02-15 11:23:00|USER007306|  LIV28859| Live Stream|             100|    105.36|
|2024-02-15 18:32:00|USER005814|  LIV85937| Live Stream|             100|    101.34|
|2024-02-15 13:01:00|USER008486|  LIV35187| Live Stream|             100|     97.15|
|2024-02-15 08:01:00|USER002750|  LIV83892| Live Stream|             100|     84.93|
|2024-02-15 14:44:00|USER008331|  LIV76007| Live Stream|             100|     77.00|
|2024-02-15 22:57:00|USER007024|  LIV86996| Live Stream|             100|     20.17|
|2024-02-15 20:56:00|USER001844|  LIV34873| Live Stream|              99|     67.04|
|2024-02-15 13:02:00|USER003599|  LIV99017| Live Stream|         

High-engagement records found: 110

=== Query 3: User Content Preferences ===


+----------+-------------------+------------+----------+--------------+
|   user_id|    engagement_date|content_type|watch_time|   device_type|
+----------+-------------------+------------+----------+--------------+
|USER000001|2024-02-16 19:07:00|       Video|     29.09|      Smart TV|
|USER000001|2024-03-23 10:43:00|     Podcast|     40.37|        Mobile|
|USER000001|2024-03-26 04:24:00|     Article|     13.69|Gaming Console|
|USER000001|2024-04-06 20:43:00|     Article|      3.23|        Mobile|
|USER000001|2024-05-19 17:06:00| Live Stream|     89.71|Gaming Console|
|USER000001|2024-06-24 00:50:00| Live Stream|     39.48|      Smart TV|
|USER000001|2024-06-25 22:51:00|     Podcast|     19.84|Gaming Console|
|USER000001|2024-07-04 08:19:00|     Article|     17.52|       Desktop|
|USER000001|2024-07-28 21:15:00|     Podcast|     23.12|        Mobile|
|USER000001|2024-09-30 20:12:00|     Article|      9.16|        Tablet|
|USER000001|2024-10-31 11:12:00|     Podcast|     44.24|        

User preference records found: 25


## Step 6: Analyze Clustering Effectiveness

### Understanding the Impact

Let's examine how liquid clustering has organized our data and analyze some aggregate statistics to demonstrate the media insights possible with this optimized structure.

### Key Analytics

- **User engagement patterns** and content preferences
- **Content performance** by type and popularity metrics
- **Device usage trends** and platform optimization
- **Time-based consumption patterns** and programming insights

In [1]:
# Analyze clustering effectiveness and media insights


# User engagement analysis

print("=== User Engagement Analysis ===")

user_engagement = spark.sql("""

SELECT user_id, COUNT(*) as total_sessions,

       ROUND(SUM(watch_time), 2) as total_watch_time,

       ROUND(AVG(watch_time), 2) as avg_session_time,

       ROUND(AVG(engagement_score), 2) as avg_engagement,

       COUNT(DISTINCT content_type) as content_types_used

FROM media.analytics.content_engagement_uf

GROUP BY user_id

ORDER BY total_watch_time DESC

LIMIT 10

""")



user_engagement.show()


# Content type performance

print("\n=== Content Type Performance ===")

content_performance = spark.sql("""

SELECT content_type, COUNT(*) as total_engagements,

       ROUND(SUM(watch_time), 2) as total_watch_time,

       ROUND(AVG(watch_time), 2) as avg_watch_time,

       ROUND(AVG(engagement_score), 2) as avg_engagement,

       COUNT(DISTINCT user_id) as unique_users,

       COUNT(DISTINCT content_id) as unique_content

FROM media.analytics.content_engagement_uf

GROUP BY content_type

ORDER BY total_watch_time DESC

""")



content_performance.show()


# Device usage analysis

print("\n=== Device Usage Analysis ===")

device_analysis = spark.sql("""

SELECT device_type, COUNT(*) as total_sessions,

       ROUND(SUM(watch_time), 2) as total_watch_time,

       ROUND(AVG(watch_time), 2) as avg_session_time,

       ROUND(AVG(engagement_score), 2) as avg_engagement,

       COUNT(DISTINCT user_id) as unique_users

FROM media.analytics.content_engagement_uf

GROUP BY device_type

ORDER BY total_watch_time DESC

""")



device_analysis.show()


# Hourly engagement patterns

print("\n=== Hourly Engagement Patterns ===")

hourly_patterns = spark.sql("""

SELECT HOUR(engagement_date) as hour_of_day, COUNT(*) as engagement_events,

       ROUND(SUM(watch_time), 2) as total_watch_time,

       ROUND(AVG(engagement_score), 2) as avg_engagement,

       COUNT(DISTINCT user_id) as active_users

FROM media.analytics.content_engagement_uf

WHERE DATE(engagement_date) = '2024-02-01'

GROUP BY HOUR(engagement_date)

ORDER BY hour_of_day

""")



hourly_patterns.show()


# Monthly engagement trends

print("\n=== Monthly Engagement Trends ===")

monthly_trends = spark.sql("""

SELECT DATE_FORMAT(engagement_date, 'yyyy-MM') as month,

       COUNT(*) as total_engagements,

       ROUND(SUM(watch_time), 2) as monthly_watch_time,

       ROUND(AVG(watch_time), 2) as avg_session_time,

       ROUND(AVG(engagement_score), 2) as avg_engagement,

       COUNT(DISTINCT user_id) as active_users

FROM media.analytics.content_engagement_uf

GROUP BY DATE_FORMAT(engagement_date, 'yyyy-MM')

ORDER BY month

""")



monthly_trends.show()

=== User Engagement Analysis ===


+----------+--------------+----------------+----------------+--------------+------------------+
|   user_id|total_sessions|total_watch_time|avg_session_time|avg_engagement|content_types_used|
+----------+--------------+----------------+----------------+--------------+------------------+
|USER007757|            40|         1786.04|           44.65|         71.45|                 4|
|USER006953|            38|         1779.15|           46.82|         71.37|                 4|
|USER009153|            32|         1766.97|           55.22|         76.38|                 4|
|USER004023|            38|         1765.27|           46.45|         72.34|                 4|
|USER011324|            40|         1751.78|           43.79|          73.4|                 4|
|USER007714|            38|         1746.63|           45.96|         71.61|                 4|
|USER005489|            38|         1731.22|           45.56|         73.16|                 4|
|USER006643|            40|         1729

+------------+-----------------+----------------+--------------+--------------+------------+--------------+
|content_type|total_engagements|total_watch_time|avg_watch_time|avg_engagement|unique_users|unique_content|
+------------+-----------------+----------------+--------------+--------------+------------+--------------+
| Live Stream|            75343|      4760788.06|         63.19|         79.91|       11906|         50968|
|     Podcast|            75155|      2630926.58|         35.01|         69.74|       11915|         50896|
|       Video|            75314|      1584940.33|         21.04|         74.64|       11910|         50942|
|     Article|            74959|       838960.65|         11.19|         64.61|       11909|         51014|
+------------+-----------------+----------------+--------------+--------------+------------+--------------+


=== Device Usage Analysis ===


+--------------+--------------+----------------+----------------+--------------+------------+
|   device_type|total_sessions|total_watch_time|avg_session_time|avg_engagement|unique_users|
+--------------+--------------+----------------+----------------+--------------+------------+
|      Smart TV|         60479|      2167853.26|           35.84|         79.49|       11789|
|Gaming Console|         60124|      2064190.85|           34.33|          75.7|       11813|
|       Desktop|         59997|      1950615.35|           32.51|         72.44|       11789|
|        Tablet|         59849|      1862777.95|           31.12|         68.52|       11771|
|        Mobile|         60322|      1770178.21|           29.35|          65.0|       11790|
+--------------+--------------+----------------+----------------+--------------+------------+


=== Hourly Engagement Patterns ===


+-----------+-----------------+----------------+--------------+------------+
|hour_of_day|engagement_events|total_watch_time|avg_engagement|active_users|
+-----------+-----------------+----------------+--------------+------------+
|          0|               13|          361.49|          67.0|          13|
|          1|                2|           60.02|          65.5|           2|
|          2|               12|          237.20|          71.0|          12|
|          3|                3|           33.81|          64.0|           3|
|          4|                3|           65.32|         72.67|           3|
|          5|                1|           28.42|          73.0|           1|
|          6|               19|          557.21|         68.79|          19|
|          7|               39|         1311.27|         74.69|          39|
|          8|               50|         1492.66|          71.4|          50|
|          9|               40|         1211.88|          72.4|          40|

+-------+-----------------+------------------+----------------+--------------+------------+
|  month|total_engagements|monthly_watch_time|avg_session_time|avg_engagement|active_users|
+-------+-----------------+------------------+----------------+--------------+------------+
|2024-01|            25469|         835801.11|           32.82|          72.3|       10295|
|2024-02|            23976|         784023.26|           32.70|         72.17|       10071|
|2024-03|            25635|         838410.60|           32.71|         72.21|       10238|
|2024-04|            24772|         802608.23|           32.40|         72.08|       10116|
|2024-05|            25101|         811778.86|           32.34|         72.31|       10201|
|2024-06|            24789|         804955.18|           32.47|         72.25|       10191|
|2024-07|            25280|         830363.38|           32.85|         72.27|       10212|
|2024-08|            25586|         831211.87|           32.49|         72.25|  

## Key Takeaways: Iceberg and Liquid Clustering in AIDP

### What We Demonstrated

1. **Automatic Optimization**: Created a table with `TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (user_id, engagement_date)` and let Delta automatically optimize data layout

2. **Performance Benefits**: Queries on clustered columns (user_id, engagement_date) are significantly faster due to data locality

3. **Zero Maintenance**: No manual partitioning, bucketing, or Z-Ordering required - Delta handles it automatically

4. **Real-World Use Case**: Media analytics where content engagement and user behavior analysis are critical

### AIDP Advantages

- **Unified Analytics**: Seamlessly integrates with other AIDP services
- **Governance**: Catalog and schema isolation for media data
- **Performance**: Optimized for both OLAP and OLTP workloads
- **Scalability**: Handles media-scale data volumes effortlessly

### Best Practices for Iceberg and Liquid Clustering

1. **Choose clustering columns** based on your most common query patterns
2. **Start with 1-4 columns** - too many can reduce effectiveness
3. **Consider cardinality** - high-cardinality columns work best
4. **Monitor and adjust** as query patterns evolve

### Next Steps

- Explore other AIDP features like AI/ML integration
- Try liquid clustering with different column combinations
- Scale up to larger media datasets
- Integrate with real content management and streaming platforms

This notebook demonstrates how Oracle AI Data Platform combines Delta's advanced liquid clustering with Iceberg's open, future-proof architecture to deliver enterprise-grade analytics that are both high-performance and standards-compliant.